In [ ]:
# Compare the f_value (measure of customers disconnected by storm, 0 -> all, 1-> none)
# for various transmission line destruction wind speed thresholds.

# Specific example: Hurricane Maria 2017, and its impacts on the Puerto Rican grid

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
import re
from glob import glob
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import geopandas as gpd

In [ ]:
def natural_sort(s):
    return sorted(s, key=lambda s: [int(t) if t.isdigit() else t.lower() for t in re.split('(\d+)', str(s))])

In [ ]:
# load our target data

# expect target files:
#   to be named "targets__<...>.gpkg"
#   to reside in a directory named with the wind speed threshold, e.g. "20"

target_files = natural_sort(
    [
        Path(p) for p in glob(os.path.join(os.getcwd(), "output", "*/**"))
        if "targets__" in p and p.endswith(".gpkg")
    ]
)

targets = {}
for path in target_files:
    wind_speed_threshold = int(path.parent.name)
    df = gpd.read_file(path)
    df.f_value = df.f_value.astype(float)
    targets[wind_speed_threshold] = df

In [ ]:
# have a look at an example target dataset

list(targets.values())[0].head()

In [ ]:
# f_value for various transmission line wind speed destruction thresholds

f, axes = plt.subplots(
    nrows=len(targets),
    figsize=(14, 8 * len(targets))
)

for i, (threshold, df) in enumerate(targets.items()):
    axes[i] = df.plot(
        column="f_value",
        ax=axes[i],
        cmap='tab20c',
        legend=True,
        legend_kwds={
            "label": "f_value (Fraction of customers with electricity)",
            "orientation": "horizontal",
            "aspect": 30,
        }
    )
    axes[i].set_title(f"f_value({threshold})")
    axes[i].set_xlabel("Longitude")
    axes[i].set_ylabel("Latitude")
    
f.savefig("f_value.pdf")

In [ ]:
# Difference between f_value for various transmission line wind speed destruction thresholds

lowest_speed, *other_speeds = sorted(targets.keys())

f, axes = plt.subplots(
    nrows=len(other_speeds),
    figsize=(14, 10 * len(other_speeds))
)

# set the colorbar range to [0, 1]
norm = Normalize(vmin=0, vmax=1)

for i, threshold in enumerate(other_speeds):
    df = targets[threshold]
    df["difference"] = df.f_value - targets[lowest_speed].f_value
    axes[i] = df.plot(
        column="difference",
        cmap='Reds',
        norm=norm,
        ax=axes[i],
        legend=True,
        legend_kwds={
            "label": "Difference",
            "orientation": "horizontal",
            "aspect": 30
        }
    )
    axes[i].set_title(f"f_value({threshold}) - f_value({lowest_speed})")
    axes[i].set_xlabel("Longitude")
    axes[i].set_ylabel("Latitude")
    
f.savefig("f_value_difference.pdf")